In [27]:
'''
臺灣大學學術期刊資料庫
http://ejournal.press.ntu.edu.tw/index.php
'''

'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 處理下拉式選單的工具
from selenium.webdriver.support.ui import Select

# 加入行為鍊 ActionChain (在 WebDriver 中模擬滑鼠移動、點繫、拖曳、按右鍵出現選單，以及鍵盤輸入文字、按下鍵盤上的按鈕等)
from selenium.webdriver.common.action_chains import ActionChains

# 取得系統時間的工具
from datetime import datetime

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# regular expression 工具
import re

# 子處理程序，用來取代 os.system 的功能
import subprocess

# 隨機亂數(整數)
from random import randint

# 格式化字串輸出
from pprint import pprint as pp

# 圖形使用者介面: 在電腦桌面模擬人類行為來控制滑鼠跟鍵籃
import pyautogui

'''
[1] Selenium with Python 中文翻譯文檔
參考網頁：https://selenium-python-zh.readthedocs.io/en/latest/index.html
[2] selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
[3] Mouse Hover Action in Selenium
參考網址：https://python-selenium-zh.readthedocs.io/zh_CN/latest/7.2%20%E8%A1%8C%E4%B8%BA%E9%93%BE/
[4] How to select a drop-down menu value with Selenium using Python?
參考網址：https://stackoverflow.com/questions/7867537/how-to-select-a-drop-down-menu-value-with-selenium-using-python
[5] PyAutoGUI : 使用Python控制電腦
https://yanwei-liu.medium.com/pyautogui-%E4%BD%BF%E7%94%A8python%E6%93%8D%E6%8E%A7%E9%9B%BB%E8%85%A6-662cc3b18b80
'''

'''
設定
'''
# 建立儲存檔案的資料夾，不存在就新增
folderPath = 'ntu_journal_db'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知
my_options.add_argument('--user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"')  #請求標頭中的使用者代理資訊

# 指定 chromedriver 檔案的路徑
my_executable_path = './chromedriver.exe'

# Web Driver 變數
driver = None

# 走訪首頁
url = "http://ejournal.press.ntu.edu.tw/index.php"

# 放置資料的變數
listData = []

# JSON 存檔路徑
json_path = "./臺灣大學學術期刊資料庫.json"


'''程式區域'''
# web driver 初始化
def init():
    global driver
    
    # 使用 Chrome 的 WebDriver
    driver = webdriver.Chrome( 
        options = my_options, 
        executable_path = my_executable_path
    )
    
# 走訪首頁
def visitHomePage():
    # 進入首頁
    driver.get(url)
    
    # 按下 中文 按鈕
    driver.find_element(By.CSS_SELECTOR, 'input[type="submit"][value="中文"]').click()

# 打開所有隱藏連結
def getSideBarLinks():
    try:
        # 等待特定元素出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "div.level_browse")
            )
        )
        
        # 建立行為鍊
        actions = ActionChains(driver)
        
        # 開啟所有子列表(尋找 a)
        a_plus_elms = driver.find_elements(By.PARTIAL_LINK_TEXT, '+')
        for a in a_plus_elms:
            actions.click(on_element=a)
        actions.perform()
         
        # 取得期刊號碼的超連結
        a_link_elms = driver.find_elements(By.CSS_SELECTOR, 'a[href^="query.php"]')
        for a_link in a_link_elms:
            listData.append({
                "title": a_link.get_attribute('innerText'),
                "link": a_link.get_attribute('href')
            })

    except TimeoutException:
        print("元素讀取逾時，即將關閉瀏覽器")
        close()
        
# 走訪各期頁面
def getMainPageLinks():     
    # 取得主要頁面的 regex
    regex_link = r"https?:\/\/.+&lvbw=\S\d{5}"
    
    # 取得篇數的 regex
    regex_total = r".+\((\d+)\)"
    
    # 預估期數與篇數
    count_title = 0
    count_total = 0
    
    for index, _dict in enumerate(listData):
        # 沒有 sub 屬性，則建立，為了放置細節頁的內容
        if "sub" not in listData[index]:
            listData[index]['sub'] = []
            
        # 可能有 PDF 資訊的網頁連結才走訪
        if re.search(regex_link, _dict['link']) != None:
            # 累加期數
            count_title += 1
            
            # 累加篇數
            num = re.search(regex_total, _dict['title'])[1]
            count_total += int(num)

            
            # 走訪每一個有 PDF 連結的頁面
            driver.get(_dict['link'])

            try:
                # 等待特定元素出現
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, "div.DataList")
                    )
                )

                # 取得所有超連結
                div_elms = driver.find_elements(By.CSS_SELECTOR, "div.DataList")
                for div in div_elms:
                    # 取得放置出處、作者、電子檔等資訊的元素
                    div_field_value_elms = div.find_elements(By.CSS_SELECTOR, "div.class_area > div.field_value")

                    # 出處
                    source = div_field_value_elms[0].get_attribute('innerText')

                    # 作者
                    author = div_field_value_elms[1].get_attribute('innerText')

                    # 連結
                    if len( div_field_value_elms[2].find_elements(By.CSS_SELECTOR, "a") ) > 0:
                        link = div_field_value_elms[2].find_element(By.CSS_SELECTOR, "a").get_attribute('href')
                    else:
                        link = None

                    # 整理資料
                    listData[index]['sub'].append({
                        "source": source,
                        "author": author,
                        "link": link
                    })
            except TimeoutException:
                print(f"連結[{_dict['link']}]找不到 div.DataList 元素")
                continue
                
            # 強制等待
            sleep( randint(2,5) )
            
# 關閉瀏覽器
def close():
    driver.quit()

# 儲存 JSON
def saveToJson():
    global listData
    with open(json_path, "w", encoding='utf-8') as fp:
        fp.write( json.dumps(listData, ensure_ascii=False, indent=4) )

# 讀取 JSON
def loadJson():
    global listData
    with open(json_path, "r", encoding="utf-8") as fp:
        listData = json.loads( fp.read() )        
    
# 測試單一頁面行為鍊功能
def test():
    # 等待
    sleep(2)
    
    # 進入頁面
    driver.get("http://ejournal.press.ntu.edu.tw/query.php?Action_From=level&lvbw=F10101")
    
    # 等待
    sleep(2)
    
    # 等待特定元素出現
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, "div.DataList")
        )
    )

    # 取得所有超連結
    div_elms = driver.find_elements(By.CSS_SELECTOR, "div.DataList")
    
    # 測試連結
    link = ''
    
    for div in div_elms:
        # 取得放置出處、作者、電子檔等資訊的元素
        div_field_value_elms = div.find_elements(By.CSS_SELECTOR, "div.class_area > div.field_value")

        # 出處
        source = div_field_value_elms[0].get_attribute('innerText')

        # 作者
        author = div_field_value_elms[1].get_attribute('innerText')

        # 連結
        if len( div_field_value_elms[2].find_elements(By.CSS_SELECTOR, "a") ) > 0:
            link = div_field_value_elms[2].find_element(By.CSS_SELECTOR, "a").get_attribute('href')
        else:
            link = None
        break


    # 進入頁面
    driver.get(link)
    
    # 等待特定元素出現
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, "div#PDF_View")
        )
    )
    
    # 建立行為鍊
    actions = ActionChains(driver)
    actions.move_by_offset(900, 205)
    actions.pause(3)
    actions.click()
    actions.perform()
    
    sleep(3)
    
    # 透過 screen 環境來按下 enter
    pyautogui.press('enter')
    
    sleep(2)
    

'''執行區域'''
if __name__ == "__main__":
    # 開始
    init()
    visitHomePage()
    getSideBarLinks()
    getMainPageLinks()
    saveToJson()

    # 測試
#     test()
    
    # 結束
    close()